In [1]:
import requests
import json

# Define admin API URLs for Alice and Bob
CENTRAL_PARTY_ADMIN_URL = "http://localhost:8001"
FL_CLIENT_ADMIN_URL = "http://localhost:8101"


def create_invitation(admin_url):
    """Create a new invitation."""
    response = requests.post(f"{admin_url}/connections/create-invitation")
    return response.json()


def receive_invitation(admin_url, invitation):
    """Receive an invitation."""
    response = requests.post(
        f"{admin_url}/connections/receive-invitation", json=invitation
    )
    return response.json()


def send_message(admin_url, connection_id, message):
    """Send a message to a connection."""
    response = requests.post(
        f"{admin_url}/connections/{connection_id}/send-message",
        json={"content": message},
    )
    return response.json()


def get_messages(admin_url, connection_id):
    """Get all messages for a connection."""
    response = requests.get(f"{admin_url}/connections/{connection_id}/messages")
    return response.json()


def get_connection(admin_url, connection_id):
    """Get the connection details."""
    response = requests.get(f"{admin_url}/connections/{connection_id}")
    return response.json()

In [2]:
# Step 1: Create an invitation from Central Party
print("Central Party: Creating invitation...")
central_party_invitation = create_invitation(CENTRAL_PARTY_ADMIN_URL)
print("Central Party's invitation:", json.dumps(central_party_invitation, indent=2))

Central Party: Creating invitation...
Central Party's invitation: {
  "connection_id": "40fe2de9-99a4-4377-beb4-c80d695ed090",
  "invitation": {
    "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation",
    "@id": "da47fea1-6fd6-4218-b35a-0fccfc18c9e7",
    "label": "Central Party Agent",
    "serviceEndpoint": "http://central-party-agent:8000",
    "recipientKeys": [
      "Hhk3rVHkVkGAnboAshuDASG9gSXoPXrU5JJFzGQsHL2W"
    ]
  },
  "invitation_url": "http://central-party-agent:8000?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZGE0N2ZlYTEtNmZkNi00MjE4LWIzNWEtMGZjY2ZjMThjOWU3IiwgImxhYmVsIjogIkNlbnRyYWwgUGFydHkgQWdlbnQiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9jZW50cmFsLXBhcnR5LWFnZW50OjgwMDAiLCAicmVjaXBpZW50S2V5cyI6IFsiSGhrM3JWSGtWa0dBbmJvQXNodURBU0c5Z1NYb1BYclU1SkpGekdRc0hMMlciXX0="
}


In [3]:
# Step 2: FL Client receives Central Party's invitation
print("FL Client: Receiving invitation...")
fl_client_connection = receive_invitation(
    FL_CLIENT_ADMIN_URL, central_party_invitation["invitation"]
)
print("FL Client's connection:", json.dumps(fl_client_connection, indent=2))

FL Client: Receiving invitation...
FL Client's connection: {
  "their_role": "inviter",
  "connection_protocol": "connections/1.0",
  "invitation_key": "Hhk3rVHkVkGAnboAshuDASG9gSXoPXrU5JJFzGQsHL2W",
  "created_at": "2024-07-05 02:48:11.220986Z",
  "routing_state": "none",
  "connection_id": "7dd230cb-49b7-41ea-9a9c-1e19f5225101",
  "their_label": "Central Party Agent",
  "request_id": "a006f40b-d8dd-4bd1-ac63-42788068d406",
  "invitation_mode": "once",
  "accept": "auto",
  "invitation_msg_id": "da47fea1-6fd6-4218-b35a-0fccfc18c9e7",
  "rfc23_state": "request-sent",
  "updated_at": "2024-07-05 02:48:11.224533Z",
  "state": "request",
  "my_did": "GFk6aBdzHXHCMSgBAXBUXP"
}


In [4]:
# Check the connection status
fl_client_connection_id = fl_client_connection["connection_id"]
fl_client_connection = get_connection(FL_CLIENT_ADMIN_URL, fl_client_connection_id)
print(
    "FL Client's connection status:",
    json.dumps(fl_client_connection["state"], indent=2),
)

central_party_connection_id = central_party_invitation["connection_id"]
central_party_connection = get_connection(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id
)
print(
    "Central Party's connection status:",
    json.dumps(central_party_connection["state"], indent=2),
)

FL Client's connection status: "request"
Central Party's connection status: "response"


In [5]:
# Step 3: Bob sends a message to Alice
print("FL Client: Sending message to Central Party...")
message_response = send_message(
    FL_CLIENT_ADMIN_URL, fl_client_connection_id, "Hello Alice, this is Bob!"
)
print("Message response:", json.dumps(message_response, indent=2))

FL Client: Sending message to Central Party...
Message response: {}


In [6]:
# Step 3: Bob sends a message to Alice
print("FL Client: Sending message to Central Party...")
message_response = send_message(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id, "Test message"
)
print("Message response:", json.dumps(message_response, indent=2))

FL Client: Sending message to Central Party...
Message response: {}
